In [1]:
from ultralytics import YOLO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import shutil
from PIL import Image

c:\Codes\ads\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_dir = Path('C:/Codes/Summer_Internship/Data')
img_path = base_dir / 'data_object_image_2' / 'training' / 'image_2'
label_path = Path('C:/Codes/Summer_Internship/Data/labels')
with open('C:/Codes/Summer_Internship/Data/classes.json', 'r') as f:
    classes = json.load(f)

classes

{'Car': 0,
 'Pedestrian': 1,
 'Van': 2,
 'Cyclist': 3,
 'Truck': 4,
 'Misc': 5,
 'Tram': 6,
 'Person_sitting': 7}

In [3]:
ims = sorted(list(img_path.glob('*')))
labels = sorted(list(label_path.glob('*')))
pairs = list(zip(ims, labels))
pairs[:2]

[(WindowsPath('C:/Codes/Summer_Internship/Data/data_object_image_2/training/image_2/000000.png'),
  WindowsPath('C:/Codes/Summer_Internship/Data/labels/000000.txt')),
 (WindowsPath('C:/Codes/Summer_Internship/Data/data_object_image_2/training/image_2/000001.png'),
  WindowsPath('C:/Codes/Summer_Internship/Data/labels/000001.txt'))]

In [4]:
train, test = train_test_split(pairs, test_size=0.1, shuffle=True)
len(train), len(test)

(6732, 749)

In [5]:
train_path = Path('train').resolve()
train_path.mkdir(exist_ok=True)
valid_path = Path('valid').resolve()
valid_path.mkdir(exist_ok=True)

In [6]:
model = YOLO('yolov8m.yaml')
model = YOLO('yolov8m.pt')

In [9]:
train_results = model.train(
    data='C:/Codes/Summer_Internship/kitti.yaml',
    epochs=10,
    patience=3,
    mixup=0.1,
    batch=10,
    project='yolov8m-kitti',
    device=0
)

Ultralytics YOLOv8.2.42  Python-3.12.4 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=C:/Codes/Summer_Internship/kitti.yaml, epochs=10, time=None, patience=3, batch=10, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=yolov8m-kitti, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

train: Scanning C:\Codes\Summer_Internship\train.cache... 7400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7400/7400 [00:00<?, ?it/s]
val: Scanning C:\Codes\Summer_Internship\valid.cache... 1417 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1417/1417 [00:00<?, ?it/s]


Plotting labels to yolov8m-kitti\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.00046875), 83 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolov8m-kitti\train4
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      6.91G      1.623      5.124      1.241         60        640:   0%|          | 3/740 [00:35<2:14:48, 10.98s/it]

In [ ]:
valid_results = model.val()

In [ ]:
preds = model.predict([test[idx][0] for idx in np.random.randint(0, len(test), (20,))], save=True)